Convert data between MNI and Talairach Spaces and create new Columns

In [148]:
import pandas as pd
import numpy as np
import importlib
import util.Data as data_util
import util.Plotly as plots

In [149]:
importlib.reload(plots)
importlib.reload(data_util)

<module 'util.Data' from 'D:\\scul\\uaic-ub\\licenta\\code\\util\\Data.py'>

# Data

In [150]:
csv_filename = r'../data_db/final_coordinates-no_conversion.csv'
columns_name = ['X(R)', 'Y(A)', 'Z(S)']
new_columns_name = ['MNIX', 'MNIY', 'MNIZ', 'TALX', 'TALY', 'TALZ']

In [151]:
df_data = data_util.import_data(csv_filename)
original_shape = df_data.shape
original_shape

(1827, 19)

In [152]:
df_data.dtypes

importance        string
author            string
title             string
table_name        string
contrast          string
keywords          string
Name              string
Left/Right        string
Broadman Area     string
X(R)               int64
Y(A)               int64
Z(S)               int64
z-score          float64
subjects           int64
cluster size     float64
p value          float64
t                float64
NV               float64
index_col          int64
dtype: object

## Split data
Split data by mni and talairach type coordinates

In [153]:
split_mask = df_data['keywords'].str.contains('MNI', regex=True)
df_mni = df_data[split_mask]
df_tal = df_data[~split_mask]

In [154]:
df_mni_coord = df_mni[columns_name]
df_tal_coord = df_tal[columns_name]

In [155]:
df_mni_coord.shape, df_tal_coord.shape

((1062, 3), (765, 3))

## Convert data

In [156]:
# MNI
df_mni2tal = data_util.process_coordinates(data=df_mni_coord, conversion_type='mni2tal', no_decimals=3)
df_mni2tal_conv = pd.concat([df_mni_coord, df_mni2tal], axis=1)
df_mni2tal_conv.columns = new_columns_name
df_mni2tal_conv.head()

,MNIX,MNIY,MNIZ,TALX,TALY,TALZ
0,-6,24,-21,-6.436,22.720,-13.440
1,6,30,-9,4.724,27.408,-2.105
2,9,27,-12,7.544,24.788,-4.990
3,3,54,-3,1.943,49.543,5.049
4,-27,-18,-18,-26.229,-16.824,-14.125


In [157]:
#Talairach
df_tal2mni = data_util.process_coordinates(data=df_tal_coord, conversion_type='tal2mni', no_decimals=3)
df_tal2mni_conv = pd.concat([df_tal2mni, df_tal_coord], axis=1)
df_tal2mni_conv.columns = new_columns_name
df_tal2mni_conv.head()

,MNIX,MNIY,MNIZ,TALX,TALY,TALZ
357,16.464,6.770,53.166,14,1,52
358,-23.050,4.589,56.033,-23,-1,54
359,37.802,-2.247,44.755,34,-7,44
360,36.464,46.588,29.523,33,40,34
361,-42.318,4.131,51.832,-41,-1,50


In [158]:
df_conv = pd.concat([df_mni2tal_conv, df_tal2mni_conv])
df_conv

,MNIX,MNIY,MNIZ,TALX,TALY,TALZ
0,-6.000,24.000,-21.000,-6.436,22.720,-13.440
1,6.000,30.000,-9.000,4.724,27.408,-2.105
2,9.000,27.000,-12.000,7.544,24.788,-4.990
3,3.000,54.000,-3.000,1.943,49.543,5.049
4,-27.000,-18.000,-18.000,-26.229,-16.824,-14.125
...,...,...,...,...,...,...
1822,-37.594,50.465,-10.098,-36.000,47.000,-2.000
1823,24.449,38.318,-7.562,22.000,35.000,0.000
1824,52.243,51.773,-1.203,48.000,47.000,7.000
1825,57.953,24.853,35.560,53.000,19.000,38.000


## Extend the original dataframe with new converted data

In [159]:
df_data = pd.concat([df_data, df_conv], axis=1)
transf_shape = df_data.shape
assert transf_shape[0] == original_shape[0] and \
       transf_shape[1] == original_shape[1] + len(new_columns_name), "Something went wrong in conversion"


# Calculate significance for all coordinates

In [160]:
df_data['significance'] = df_data.apply(data_util.significance, axis=1)
assert df_data.shape[0] == transf_shape[0] and \
       df_data.shape[1] == transf_shape[1] + 1, "Something went wrong in adding significance"
assert df_data['significance'].isna().sum() == 0
transf_shape = df_data.shape

# Save the new dataframe

In [161]:
csv_final_path = r'../data_db/final_coordinates-conversion-v2.csv'
df_data.to_csv(csv_final_path,index = False,header=True)
